In [1]:
!pip install plac

In [2]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
import pickle
from tqdm import tqdm
import json

In [9]:
import json
import re

# Data Preprocessing steps, converting data into proper format to feed into ner model updation

def data_preprocessing(json_filepath):
    training_data = []
    lines=[]
    with open(json_filepath, 'r', encoding='utf-8') as f:
    # lns = f.readlines()
    # with open(json_filepath, 'r') as f:
        lns = f.readlines()

    for ln in lns:
        resume_data = json.loads(ln)
        text = resume_data['content'].replace("\n", " ")
        entities = []
        data_annot = resume_data['annotation']
        if data_annot is not None:
            for annot in data_annot:
                #only a single point in text annotation.
                point = annot['points'][0]
                labels = annot['label']
                # handle both list of labels or a single label.
                if not isinstance(labels, list):
                    labels = [labels]

                for lbl in labels:
                    pnt_start = point['start']
                    pnt_end = point['end']
                    pnt_text = point['text']

                    lstrip_diff = len(pnt_text) - len(pnt_text.lstrip())
                    rstrip_diff = len(pnt_text) - len(pnt_text.rstrip())
                    if lstrip_diff != 0:
                        pnt_start = pnt_start + lstrip_diff
                    if rstrip_diff != 0:
                        pnt_end = pnt_end - rstrip_diff
                    entities.append((pnt_start, pnt_end + 1 , lbl))
        training_data.append((text, {"entities" : entities}))
    return training_data

def cure_entity(data: list) -> list:
    """clears out leading and trailing white spaces from entity spans.

    Args:
        data (list): json data that is intended to be cured.

    Returns:
        list: The cured data.
    """
    invalid_tokens = re.compile(r'\s')

    cleaned_data = []
    for txt, annot in data:
        entities = annot['entities']
        vld_ent = []
        for start, end, label in entities:
            vld_start = start
            vld_end = end
            while vld_start < len(txt) and invalid_tokens.match(txt[vld_start]):
                vld_start += 1
            while vld_end > 1 and invalid_tokens.match(txt[vld_end - 1]):
                vld_end -= 1
            vld_ent.append([vld_start, vld_end, label])
        cleaned_data.append([txt, {'entities': vld_ent}])
    return cleaned_data

In [29]:

def read_labels_from_json(json_filepath):
    labels_array = []
    
    with open(json_filepath, 'r', encoding='utf-8') as f:
    # lns = f.readlines()
    # with open(json_filepath, 'r') as f:
        lns = f.readlines()

    for ln in lns:
        resume_data = json.loads(ln)
        text = resume_data['content'].replace("\n", " ")
        entities = []
        annotations = resume_data['annotation']
        if annotations is not None:
          # Create an array to hold the labels

            for annotation in annotations:
                label = annotation['label']
                labels_array.extend(label)  # Add all labels (extend handles lists)

            return labels_array

# Example usage
labels_array = read_labels_from_json(r"Entity Recognition in Resumes.json")

print("Labels Array:")
print(list(set(labels_array)))


Labels Array:
['Graduation Year', 'Skills', 'College Name', 'Designation', 'Email Address', 'Location', 'Name', 'Companies worked at']


In [11]:
TRAIN_DATA = cure_entity(data_preprocessing(r"Entity Recognition in Resumes.json"))
TRAIN_DATA[0]

print(f"Training data consist of {len(TRAIN_DATA)} manually labelled resume's.")


Training data consist of 220 manually labelled resume's.


In [17]:
model = None
output_dir=Path(r"models/en_tech_resume_ner_model")
n_iter=100

In [13]:
#load the model

if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

#set up the pipeline

if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe('ner', last=True)
else:
    ner = nlp.get_pipe('ner')

Created blank 'en' model


In [14]:
from spacy.training.example import Example

i=0
for _, annotations in TRAIN_DATA:
  # print(i)
  print(annotations)
  for ent in annotations.get('entities'):
    try:
      ner.add_label(ent[2])
      i=i+1
    except Exception as e:
      print("******************")
      print(e)
      print("******************")

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()

    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for batch in spacy.util.minibatch(TRAIN_DATA, size=2):
            for text, annotations in batch:
              try:
                # create Example
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                # Update the model
                nlp.update([example], losses=losses, drop=0.3)
              except Exception as e:
                pass
                # print("****** Exception in updating model",e)
        print(losses)
  

{'entities': [[1296, 1622, 'Skills'], [993, 1154, 'Skills'], [939, 957, 'College Name'], [883, 905, 'College Name'], [856, 860, 'Graduation Year'], [771, 814, 'College Name'], [727, 769, 'Designation'], [407, 416, 'Companies worked at'], [372, 405, 'Designation'], [95, 145, 'Email Address'], [60, 69, 'Location'], [49, 58, 'Companies worked at'], [13, 46, 'Designation'], [0, 12, 'Name']]}
{'entities': [[1155, 1199, 'Email Address'], [743, 1141, 'Skills'], [729, 733, 'Graduation Year'], [675, 702, 'College Name'], [631, 673, 'Degree'], [625, 629, 'Graduation Year'], [614, 623, 'College Name'], [606, 612, 'Degree'], [104, 148, 'Email Address'], [62, 68, 'Location'], [0, 14, 'Name']]}
{'entities': [[3749, 3757, 'Skills'], [3709, 3718, 'Skills'], [3664, 3672, 'Skills'], [3636, 3645, 'Skills'], [3542, 3550, 'Skills'], [3527, 3530, 'Skills'], [3510, 3515, 'Skills'], [3489, 3498, 'Skills'], [3468, 3478, 'Skills'], [3421, 3458, 'College Name'], [3381, 3419, 'Degree'], [1664, 1673, 'Location'], 

d:\TY\resume-parser-nlp\.venv\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Shraddha Achar Mathura, Uttar Pradesh - Email me o..." with entities "[[975, 1020, 'Skills'], [814, 842, 'College Name']...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
d:\TY\resume-parser-nlp\.venv\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Jacob Philip Kottayam, Kerala - Email me on Indeed..." with entities "[[2165, 2214, 'Skills'], [2145, 2155, 'Degree'], [...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
d:\TY\resume-parser-nlp\.venv\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Som

{'ner': 11964.897506285235}
{'ner': 4168.089831784017}
{'ner': 3779.480875767823}
{'ner': 3201.0629807810124}
{'ner': 2708.7795409799596}
{'ner': 2872.70802408677}
{'ner': 2290.6793747029724}
{'ner': 2207.2862627273585}
{'ner': 2214.855951428958}
{'ner': 2056.3100411100904}
{'ner': 1922.7313955281888}
{'ner': 1935.4004945244308}
{'ner': 1726.431781087183}
{'ner': 1723.917079017387}
{'ner': 1743.3981641804467}
{'ner': 1616.3132772903612}
{'ner': 1666.3575686940198}
{'ner': 1536.7866582114896}
{'ner': 1533.509531465309}
{'ner': 1566.9202219128165}
{'ner': 1469.9203109263517}
{'ner': 1359.9419249253358}
{'ner': 1334.3345710142846}
{'ner': 1386.6131669133817}
{'ner': 1381.6180069793215}
{'ner': 1287.460119908108}
{'ner': 1287.2619467951238}
{'ner': 1241.7658209226909}
{'ner': 1127.5832213467122}
{'ner': 1152.6627416159167}
{'ner': 1193.9854044375147}
{'ner': 1221.0118290448347}
{'ner': 1113.8568071922261}
{'ner': 1216.1874674858548}
{'ner': 1181.799164784178}
{'ner': 1139.4625909771696}
{'

In [15]:
for text, _ in TRAIN_DATA:
  print(text)
  doc = nlp(text)
  print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
  break

Pankaj Bhosale Microsoft SQL-SERVER  Dhule, Maharashtra - Email me on Indeed: indeed.com/r/Pankaj-Bhosale/2d6f2e970b9a7ff6  WORK EXPERIENCE  Microsoft SQL-SERVER  Technical Expertise:- Environment Windows 7, Windows XP, Linux. Languages C, C++, C#.NET. Web Technologies ASP.NET 3.5, Ajax. Databases Microsoft SQL-SERVER 2005/8, MySql, MS-ACESS.. Scripting Language HTML, JavaScript, jQuery. Integrated Development Environment Visual Studio 2008/2010.  Jr. Developer  Softzel Technologies Pvt. Ltd -  Pune, Maharashtra  Database designing, User side coding & user side designing Technologies Used ASP.NET, C#.NET, SQL Server 2005 Responsibilities ⇨ Can get online help to designers, engineers and Vaasthu experts for the construction of new houses or buildings. ⇨ Develop this site to get the list of contractors, engineers and hardware dealers.  Grocery Shop System -  Pune, Maharashtra  Pune Technologies Used Microsoft Visual Studio .Net 2008 Responsibilities ⇨ Maintain daily purchase & sales acti

In [20]:
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

Saved model to models\en_tech_resume_ner_model


In [26]:
model = spacy.load(output_dir)
resume_1 = """
Om Singh  
johndoe@example.com  
+1-555-987-1234  
San Francisco, CA  

PROFESSIONAL SUMMARY  
Experienced Full-Stack Developer with 5+ years of expertise in developing scalable web applications. Proficient in Python, JavaScript, and cloud services such as AWS and GCP.

EXPERIENCE  
Software Engineer at Tech Solutions  
June 2019 - Present  
- Developed responsive web applications using Django, React.js, and Node.js.  
- Built and optimized cloud infrastructure on AWS and Google Cloud Platform.  
- Integrated with MongoDB, PostgreSQL, and Redis for database management.  
- Implemented CI/CD pipelines with Jenkins and Docker for automated deployments.

EDUCATION  
B.S. in Computer Science  
University of California, Berkeley  
Graduated: May 2018  

SKILLS  
Python, JavaScript, Java, Go  
Django, React.js, Nodejs Node 
MongoDB, PostgreSQL, Redis  
AWS, Google Cloud Platform  
Jenkins, Docker, Kubernetes  

EXTERNAL LINKS  
https://github.com/johndoe  
https://linkedin.com/in/johndoe  
"""

In [27]:
doc2= nlp(resume_1)
print_arr = [ (ent.text, ent.label_) for ent in doc2.ents]
print('Entities', print_arr)
# doc2.ents

Entities [('Software Engineer', 'Designation'), ('B.S. in Computer Science', 'Degree'), ('Python, JavaScript, Java, Go  \nDjango, React.js, Nodejs Node \nMongoDB, PostgreSQL, Redis  \nAWS, Google Cloud Platform  \nJenkins, Docker, Kubernetes  \n\nEXTERNAL LINKS  \nhttps://github.com/johndoe  \nhttps://linkedin.com/in/johndoe  \n', 'Skills')]
